# 파이썬으로 구글 뉴스 크롤링해보기
---
단순히 뉴스레터 구독을 위한 시스템을 만들고 싶어도, 뉴스 데이터를 이용해서 분석을 하고 싶을때도 뉴스기사 크롤링은 중요한 테크닉입니다. 그런데, 특정 언론사의 정보만 수집하는게 아니라 다양한 언론사의 정보를 모두 모으고 싶다면 크롤링 난이도가 높아지기 때문에 쉽게 도전하기는 어려운 주제지요. 이번에는 파이썬(Python)에서 이용할 수 있는 셀레니움(Selenium), 뷰티풀수프(BeautifulSoup), 뉴스페이퍼(newspaper3k) 패키지를 이용해 구글 뉴스 정보를 크롤링하도록 하겠습니다.
</p></br></br>

## URL 특징 알아보기
---
웹사이트 크롤링을 할 때는 URL의 특징을 알아보면 신경쓸 일이 적어집니다. 구글 뉴스 검색 결과의 URL은 검색어와 페이지, 검색 기간이 나타나는 구조이기 때문에 적당히 문자열 파싱을 해 주면 좋습니다.
</p></br></br>

`https://www.google.com/search?q=(검색어)&sca_esv=1814fa2a4600643d&tbas=0&tbs=qdr:m&tbm=nws&ei=rE3pZeLxNeHX1e8PpdOcMA&start=(페이지 정보)&sa=N&ved=2ahUKEwji9-zrsuGEAxXha_UHHaUpBwYQ8tMDegQIBBAE&biw=2560&bih=1313&dpr=1`
</p></br></br>

위 URL을 살펴보면, **(검색어)** 부분에 필요한 키워드를 집어넣으면 되고, **(페이지 정보)** 부분에 검색을 원하는 페이지를 넣으면 됩니다. 이 때, 페이지 정보는 `0이 1쪽, 10이 2쪽, 20이 3쪽...` 과 같은 규칙을 띠고 있습니다.
</p></br></br>


In [1]:
# import package
from bs4 import BeautifulSoup
import requests
import urllib
import re
import pandas as pd
from newspaper import Article
from selenium import webdriver

keyword = '아마존닷컴'
page_num = 0  # 0, 10, 20...
news_df = pd.DataFrame()

# selenium headless mode
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(options=options)

</p></br></br>

## 구글 뉴스 제목 크롤링
---
필요한 패키지와 변수를 설정한 뒤, 크롤링을 할 수 있도록 코드를 작성합니다. 기본적으로는 뷰티풀수프를 이용해 구글 뉴스 검색 결과를 리스트업합니다. 각 기사의 제목은 `div.BNeawe.vvjwJb` 구역에 저장되어 있으므로 해당 내용을 저장해 줍니다. 그리고, 각 기사의 링크는 a 태그를 이용해서 지정되어 있으므로, 페이지의 a 태그 중 기사의 제목과 일치하는 항목의 `href` 요소를 저장하는 방식으로 구현했습니다. 이렇게 수집된 URL은 쓸모없는 요소가 많으므로, 필요한 영역을 제외하고는 삭제하도록 합니다.
</p></br></br>

`/url?q=https://www.etnews.com/20240229000171&sa=U&ved=2ahUKEwi-xNzLquOEAxXnna8BHQfrC-8QxfQBegQIABAC&usg=AOvVaw2seH5DXmAoe19lxO12URlk`
</p></br></br>

예를 들어, 한 URL은 위와 같은 구성을 띠고 있습니다. 여기서 `/url?q=` 부분과 `&sa=...` 부분은 쓸모없으므로 `re.findall()` 함수를 이용해 적절한 정규식을 작성해 삭제해 주도록 합니다. 저는 `http\S+&sa` 형태로 만들어서 http로 시작하는 주소 영역만 저장하도록 했습니다. 이 때, `urllib.parse.unquote()` 함수를 이용해 인코딩 정보를 맞춰주는 작업을 진행합니다.
</p></br></br>

## 구글 뉴스 본문 크롤링
---
다음으로는 뉴스 본문입니다. 구글 검색결과에는 요약본만 제공되기 때문에, 전체 본문 크롤링을 위해 각 언론사 홈페이지에 접속하는 방식을 구현했습니다. 각 홈페이지의 구조에 맞춰 크롤러를 만드는건 사실상 불가능하기 때문에, `newspaper` 패키지를 이용해 데이터를 저장하도록 했습니다. 그리고, 간혹 접속할 때 SSL 에러가 발생하는 경우가 확인되어 에러가 발생할 경우 셀레니움을 이용해 접속하는 작업을 추가했습니다.
</p></br></br>


In [ ]:
while 1:
    # google news crawling
    url = f'https://www.google.com/search?q={keyword}&sca_esv=1814fa2a4600643d&tbas=0&tbs=qdr:m&tbm=nws&ei=rE3pZeLxNeHX1e8PpdOcMA&start={page_num}&sa=N&ved=2ahUKEwji9-zrsuGEAxXha_UHHaUpBwYQ8tMDegQIBBAE&biw=2560&bih=1313&dpr=1'
    req = requests.get(url)
    content = req.content
    soup = BeautifulSoup(content, 'html.parser')

    # last page check
    if soup.select('div.BNeawe.vvjwJb') == []: break

    title_list = [t.text for t in soup.select('div.BNeawe.vvjwJb')]  # title
    url_list = []

    # url
    for u in soup.select('a'):
        for t in title_list:
            if t in u.text:
                temp_url = urllib.parse.unquote(u['href'])
                temp_url = re.findall('http\S+&sa',temp_url)[0][:-3]
                url_list.append(temp_url)

    # article
    for ind, news_url in enumerate(url_list):
        try:
            article = Article(url=news_url)
            article.download()
            article.parse()    
            news_article = article.text
        except:  # ssl error
            driver.get(news_url)
            article.download(input_html=driver.page_source)
            article.parse()
            news_article = article.text

        news_df = pd.concat([news_df, pd.DataFrame([[title_list[ind], news_article, news_url]])])
    
    news_df[0] = news_df[0].apply(lambda x: re.sub('\s+',' ',x))
    news_df = news_df.reset_index(drop=True)

    page_num += 10
    news_df.to_excel('news_crawling.xlsx', index=False)

news_df.columns = ['본문','URL','제목']

</p></br></br>

## 저장된 데이터 확인
---
저장된 데이터는 파이썬의 `Pandas` 패키지를 이용해 표 형식으로 만들어 줍니다. 본문, URL, 제목 항목을 저장한 뒤 xlsx 파일로 내보내기를 해 줬습니다. 이후에는 크롤링한 데이터를 이용해 자연어 분석을 해볼 수 있겠네요.
</p></br></br>


In [10]:
news_df.head(1)

,본문,URL,제목
0,아마존닷컴(AMZN) 52주 신고가,아마존닷컴( AMZN )가 12일(현지시각) 전일 대비 2.71% 오른 174.64...,https://www.hankyung.com/article/202402128580R
